<a href="https://colab.research.google.com/github/kirath2205/Machine-Learning-Plant-seedling/blob/main/inception_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [8]:
def define_model(width, height):
    
    # define the input to the model
    input_model = tf.keras.layers.Input(shape = (width, height, 3), name = 'image_input')
    
    # main model with Incpetion - ResNet - v2 layers
    # omit the top layers as we are adding custom layers
    # use transfer learning, with weights from Imagenet trained model
    main_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top = False, weights = 'imagenet')(input_model)
    
    # flatten model to get appropriate dimensions
    flattened_model = tf.keras.layers.Flatten()(main_model)
    
    # add custom dropout and dense layers
    dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
    dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    
    # output of model
    output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

    model = tf.keras.models.Model(input_model,  output_model)
    
    # use Adam optimizer with model
    optimizer = tf.keras.optimizers.Adam(lr = 5e-4, beta_1 = 0.9, beta_2 = 0.999)
    
    # use categorical crossentropy loss since classification task
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model


In [17]:
def training_callbacks():
    
    # save best model regularly
    save_best_model =  tf.keras.callbacks.ModelCheckpoint('best_model_inception_resnet_plant_seedlings_attempt_1', monitor='val_accuracy', verbose=1, save_best_only=True)
    
    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor = 0.8,
                              patience = 4, min_lr = 1e-7, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'loss', min_delta = 1e-2, patience = 15, verbose = 1,
        mode = 'min', baseline = None, restore_best_weights = True
    )

    return [save_best_model, reduce_lr, early_stopping]

In [21]:
def define_generators():
  height = 299
  width = 299
  batch_size = 32
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=360,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.5,
        vertical_flip=True,
        horizontal_flip=True,
        validation_split=0.1, # change to use validation instead of training on entire training set
    )

  train_generator = train_datagen.flow_from_directory(
        directory='Documents/train',
        target_size=(width, height),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode="categorical",
        subset='training',
    )

  validation_generator = train_datagen.flow_from_directory(
        directory='Documents/train',
        target_size=(width, height),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode="categorical",
        subset='validation',
    )

  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

  test_generator = test_datagen.flow_from_directory(
        directory='Documents/',
        classes=['test'],
        target_size=(width, height),
        batch_size=1,
        color_mode='rgb',
        shuffle=False,
        class_mode='categorical')

  return train_generator, validation_generator, test_generator


In [22]:
height = 299
width = 299
num_epochs = 200
batch_size = 32

In [ ]:
model = define_model(width, height)
train_gen, validation_gen, test_gen = define_generators()

# the actual training
history = model.fit(
    train_gen,
    callbacks = training_callbacks(),
    epochs = num_epochs,
    steps_per_epoch = train_gen.samples // batch_size,
    validation_data = validation_gen,
    validation_steps = validation_gen.samples // batch_size,
)

Found 4279 images belonging to 12 classes.
Found 471 images belonging to 12 classes.
Found 794 images belonging to 1 classes.
Epoch 1/200
133/133 [==============================] - ETA: 0s - loss: 2.5908 - accuracy: 0.1253
Epoch 00001: val_accuracy improved from -inf to 0.13616, saving model to best_model_inception_resnet_plant_seedlings_attempt_1
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: best_model_inception_resnet_plant_seedlings_attempt_1/assets
133/133 [==============================] - 142s 1s/step - loss: 2.5908 - accuracy: 0.1253 - val_loss: 2.4748 - val_accuracy: 0.1362 - lr: 5.0000e-04
Epoch 2/200
 64/133 [=============>................] - ETA: 36s - loss: 2.4720 - accuracy: 0.1388

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (9,6))
plt.plot(history.history['val_loss'],label='Test loss')
plt.plot(history.history['loss'],label='Train loss')
plt.title('Loss curve for improved vgg16 model')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.figure(figsize = (9,6))
plt.plot(history.history['val_accuracy'],label = 'Test accuracy')
plt.plot(history.history['accuracy'],label = 'Train accuracy')
plt.title('Accuracy curve for improved vgg16 model')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
model = tf.keras.models.load_model('best_model_inception_resnet_plant_seedlings_attempt_1')
print(len(test_generator))
seedlings_types = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]
predictions = model.predict(test_generator, steps=test_generator.samples)

#predictions = model.predict(test_generator, steps=test_generator.samples)

class_list = []

for i in range(0, predictions.shape[0]):
  y_class = predictions[i, :].argmax(axis=-1)
  class_list += [seedlings_types[y_class]]

submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list

submission.to_csv('inception_resnet_submission_attempt_1.csv', index=False)